In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path


## Loading Result Data

In [ ]:
from scripts.load_cases import load_case1_batches, load_caseX_batches
from scripts.flatten_cases import flatten_all_cases
flatten_all_cases()


In [ ]:
# plot output directory and save function
def save_plot(fig, name):
    output_dir = Path("Plots_AlCSL5")
    output_dir.mkdir(exist_ok=True)
    fig.savefig(output_dir / f"{name}.jpg", dpi=300, bbox_inches='tight')


In [ ]:
# loading case 1 - TAPSim only
print("Loading Case 1 (TAPSim only)...")
case1 = load_case1_batches()
print(f"Loaded {len(case1)} atom events from Case 1.")


In [ ]:
print("Loading Case 2 (TAPSim + MD Relaxation, 5-atoms interval)...")
case2_dir = '../6_simulationAnalysis/Case2_CSL5'
case2 = load_caseX_batches(case2_dir)
print(f"Loaded {len(case2)} atom events from Case 2.")


In [ ]:
print("Loading Case 3 (TAPSim + MD Relaxation, 10-atoms interval)...")
case3_dir = '../6_simulationAnalysis/Case3_CSL5'
case3 = load_caseX_batches(case3_dir)
print(f"Loaded {len(case3)} atom events from Case 3.")


## Based on Coordinates

### Coordinate Overlap:

In [ ]:
# Compute Jaccard similarities - sliding window only
from scripts.coord_overlap import (
    compute_sliding_window_jaccard,
    compute_sliding_window_3way_jaccard
)

# Sliding window Jaccard similarities
window_sizes = [100, 500]
sliding_jaccard_12 = compute_sliding_window_jaccard(case1, case2, window_sizes)
sliding_jaccard_13 = compute_sliding_window_jaccard(case1, case3, window_sizes)
sliding_jaccard_23 = compute_sliding_window_jaccard(case2, case3, window_sizes)
#sliding_jaccard_123 = compute_sliding_window_3way_jaccard(case1, case2, case3, window_sizes)

print(f"Computed Jaccard similarities for window sizes: {window_sizes}")


In [ ]:
def plot_sliding_window_jaccard_simple(sliding_data, pair_names, window_sizes):
    """Plot sliding window Jaccard similarities - simplified with fewer subplots"""
    n_pairs = len(pair_names)
    n_windows = len(window_sizes)
    
    fig, axes = plt.subplots(n_pairs, n_windows, figsize=(8*n_windows, 4*n_pairs), 
                            sharex=True, sharey=True)
    
    if n_pairs == 1:
        axes = axes.reshape(1, -1)
    if n_windows == 1:
        axes = axes.reshape(-1, 1)
    
    # Use simple colors
    colors = ['blue', 'red', 'green']
    
    for i, (data, pair_name, color) in enumerate(zip(sliding_data, pair_names, colors)):
        for j, window_size in enumerate(window_sizes):
            if window_size in data and data[window_size]:
                steps = sorted(data[window_size].keys())
                values = [data[window_size][s] for s in steps]
                
                axes[i, j].plot(steps, values, color=color, marker='o', markersize=6, linewidth=2)
                axes[i, j].set_title(f"{pair_name}\n(Window: {window_size} Atoms Evaporated)")
                #axes[i, j].grid(True, alpha=0.3)
                axes[i, j].set_ylim(-0.05, 1.05)
                
                if i == n_pairs - 1:
                    axes[i, j].set_xlabel("Number of Atoms Evaporated")
                if j == 0:
                    axes[i, j].set_ylabel("Jaccard Similarity")
    
    plt.suptitle("Sliding Window Jaccard Coordinate Overlap", fontsize=16)
    plt.tight_layout()
    return fig

# Simplified plot - only 3 comparisons instead of 4
pair_names = [
    "TAPSim (no MD) vs TAPSim with 5-atom MD",
    "TAPSim (no MD) vs TAPSim with 10-atom MD", 
    "TAPSim with 5-atom MD vs TAPSim with 10-atom MD"
]

sliding_data = [sliding_jaccard_12, sliding_jaccard_13, sliding_jaccard_23]

fig = plot_sliding_window_jaccard_simple(sliding_data, pair_names, window_sizes)
save_plot(fig, "jaccard_overlap")
plt.show()

# Print summary statistics
print("\nSliding Window Jaccard Summary:")
print("=" * 50)
for i, (data, pair_name) in enumerate(zip(sliding_data, pair_names)):
    print(f"\n{pair_name}:")
    for window_size in window_sizes:
        if window_size in data and data[window_size]:
            values = list(data[window_size].values())
            avg_jaccard = np.mean(values)
            max_jaccard = np.max(values)
            min_jaccard = np.min(values)
            print(f"  Window {window_size:3d}: Avg={avg_jaccard:.4f}, Max={max_jaccard:.4f}, Min={min_jaccard:.4f}")


### Euclidean Norm of Atom Coordinates

In [ ]:
from matplotlib import axes
import pandas as pd
import matplotlib.cm as cm

def compute_coordinate_euclidean_norm(case_data):
    """Compute Euclidean norm of atom coordinates for each step"""
    norms = {}
    for step in sorted(case_data.keys()):
        df = case_data[step]
        if len(df) > 0:
            coords = np.array([df['x'].iloc[0], df['y'].iloc[0], df['z'].iloc[0]])
            norm = np.linalg.norm(coords)
            norms[step] = norm
    return norms

def moving_average(data, window_size=20):
    return pd.Series(data).rolling(window=window_size, min_periods=1, center=True).mean().values

def plot_coordinate_euclidean_differences(case1, case2, case3, window_size=20):
    """Plot coordinate differences as Euclidean norms with moving average"""
    norms_case1 = compute_coordinate_euclidean_norm(case1)
    norms_case2 = compute_coordinate_euclidean_norm(case2)
    norms_case3 = compute_coordinate_euclidean_norm(case3)
    common_steps = sorted(set(norms_case1.keys()) & set(norms_case2.keys()) & set(norms_case3.keys()))
    diff_12 = [abs(norms_case1[step] - norms_case2[step]) for step in common_steps]
    diff_13 = [abs(norms_case1[step] - norms_case3[step]) for step in common_steps]
    diff_23 = [abs(norms_case2[step] - norms_case3[step]) for step in common_steps]

    # Compute moving averages
    ma_12 = moving_average(diff_12, window_size)
    ma_13 = moving_average(diff_13, window_size)
    ma_23 = moving_average(diff_23, window_size)

    fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    
    # Plot 1: TAPSim (no MD) vs TAPSim (5-atom MD)
    axes[0].plot(common_steps, diff_12, color='blue', marker='o', markersize=2, alpha=0.3, label='Raw')
    axes[0].plot(common_steps, ma_12, color='blue', linewidth=2, label=f'MA-{window_size}')
    axes[0].set_title("Coordinate Euclidean Norm Difference: TAPSim (no MD) vs TAPSim with 5-atom MD)")
    axes[0].set_ylabel("Distance Difference (Å)")
    #axes[0].grid(True, alpha=0.3)
    axes[0].legend()

    # Plot 2: TAPSim (no MD) vs TAPSim (10-atom MD)
    axes[1].plot(common_steps, diff_13, color='red', marker='s', markersize=2, alpha=0.3, label='Raw')
    axes[1].plot(common_steps, ma_13, color='red', linewidth=2, label=f'MA-{window_size}')
    axes[1].set_title("Coordinate Euclidean Norm Difference: TAPSim (no MD) vs TAPSim with 10-atom MD")
    axes[1].set_ylabel("Distance Difference (Å)")
#    axes[1].grid(True, alpha=0.3)
    axes[1].legend()

    # Plot 3: TAPSim (5-atom MD) vs TAPSim (10-atom MD)
    axes[2].plot(common_steps, diff_23, color='green', marker='^', markersize=2, alpha=0.3, label='Raw')
    axes[2].plot(common_steps, ma_23, color='green', linewidth=2, label=f'MA-{window_size}')
    axes[2].set_title("Coordinate Euclidean Norm Difference: TAPSim with 5-atom MD vs TAPSim with 10-atom MD")
    axes[2].set_xlabel("Number of Atoms Evaporated")
    axes[2].set_ylabel("Distance Difference (Å)")
    #axes[2].grid(True, alpha=0.3)
    axes[2].legend()
    
    plt.tight_layout()
    save_plot(fig, "Euclidean")
    plt.show()
    
    # Calculate and print summary statistics
    print("\nCoordinate Euclidean Norm Difference Summary:")
    print("=" * 60)
    
    pairs = [
        ("TAPSim (no MD) vs TAPSim with 5-atom MD", diff_12, ma_12),
        ("TAPSim (no MD) vs TAPSim with 10-atom MD", diff_13, ma_13),
        ("TAPSim with 5-atom MD vs TAPSim with 10-atom MD", diff_23, ma_23)
    ]
    
    for pair_name, raw_data, ma_data in pairs:
        print(f"\n{pair_name}:")
        print(f"  Raw Data     - Mean: {np.mean(raw_data):.3f} Å, Std: {np.std(raw_data):.3f} Å, Max: {np.max(raw_data):.3f} Å")
        print(f"  MA-{window_size} Data  - Mean: {np.mean(ma_data):.3f} Å, Std: {np.std(ma_data):.3f} Å, Max: {np.max(ma_data):.3f} Å")
    
    # Calculate trend analysis using linear regression
    from scipy import stats
    print("\nTrend Analysis (Linear Regression on Moving Averages):")
    print("-" * 50)
    
    for pair_name, _, ma_data in pairs:
        x_data = np.array(common_steps)
        slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, ma_data)
        print(f"\n{pair_name}:")
        print(f"  Slope: {slope:.6f} Å/step")
        print(f"  R²: {r_value**2:.4f}")
        print(f"  Trend: {'Increasing' if slope > 0 else 'Decreasing' if slope < 0 else 'Flat'}")
        if abs(slope) > 0.001:
            print(f"  Change over 1000 steps: {slope * 1000:.3f} Å")

plot_coordinate_euclidean_differences(case1, case2, case3, window_size=50)


In [ ]:
def compute_center_of_mass_trajectory(case_data, window_size=10):
    """Compute center of mass of evaporated atoms over sliding windows"""
    steps = sorted(case_data.keys())
    com_trajectory = {}
    
    for i in range(len(steps) - window_size + 1):
        window_steps = steps[i:i + window_size]
        x_coords, y_coords, z_coords = [], [], []
        
        for step in window_steps:
            df = case_data[step]
            if len(df) > 0:
                x_coords.append(df['x'].iloc[0])
                y_coords.append(df['y'].iloc[0])
                z_coords.append(df['z'].iloc[0])
        
        if x_coords:  # If we have data
            com_x = np.mean(x_coords)
            com_y = np.mean(y_coords)
            com_z = np.mean(z_coords)
            com_trajectory[steps[i]] = np.array([com_x, com_y, com_z])
    
    return com_trajectory

def plot_center_of_mass_divergence(case1, case2, case3, window_size=10):
    """Plot how center of mass of evaporation diverges between cases"""
    # Compute center of mass trajectories
    com1 = compute_center_of_mass_trajectory(case1, window_size)
    com2 = compute_center_of_mass_trajectory(case2, window_size)
    com3 = compute_center_of_mass_trajectory(case3, window_size)
    
    # Find common steps
    common_steps = sorted(set(com1.keys()) & set(com2.keys()) & set(com3.keys()))
    
    # Compute distances between centers of mass
    dist_12 = [np.linalg.norm(com1[step] - com2[step]) for step in common_steps]
    dist_13 = [np.linalg.norm(com1[step] - com3[step]) for step in common_steps]
    dist_23 = [np.linalg.norm(com2[step] - com3[step]) for step in common_steps]
    
    # Compute moving averages
    ma_window = 20
    ma_12 = moving_average(dist_12, ma_window)
    ma_13 = moving_average(dist_13, ma_window)
    ma_23 = moving_average(dist_23, ma_window)
    
    # Plot
    fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    
    # Plot 1: TAPSim (no MD) vs TAPSim (5-atom MD)
    axes[0].plot(common_steps, dist_12, color='blue', marker='o', markersize=2, alpha=0.3, label='Raw')
    axes[0].plot(common_steps, ma_12, color='blue', linewidth=2, label=f'MA-{ma_window}')
    axes[0].set_title("Center of Mass Distance: TAPSim (no MD) vs TAPSim with 5-atom MD)")
    axes[0].set_ylabel("Distance (Å)")
    #axes[0].grid(True, alpha=0.3)
    axes[0].legend()

    # Plot 2: TAPSim (no MD) vs TAPSim (10-atom MD)
    axes[1].plot(common_steps, dist_13, color='red', marker='s', markersize=2, alpha=0.3, label='Raw')
    axes[1].plot(common_steps, ma_13, color='red', linewidth=2, label=f'MA-{ma_window}')
    axes[1].set_title("Center of Mass Distance: TAPSim (no MD) vs TAPSim with 10-atom MD")
    axes[1].set_ylabel("Distance (Å)")
    #axes[1].grid(True, alpha=0.3)
    axes[1].legend()

    # Plot 3: TAPSim (5-atom MD) vs TAPSim (10-atom MD)
    axes[2].plot(common_steps, dist_23, color='green', marker='^', markersize=2, alpha=0.3, label='Raw')
    axes[2].plot(common_steps, ma_23, color='green', linewidth=2, label=f'MA-{ma_window}')
    axes[2].set_title("Center of Mass Distance: TAPSim with 5-atom MD vs TAPSim with 10-atom MD")
    axes[2].set_xlabel("Number of Atoms Evaporated")
    axes[2].set_ylabel("Distance (Å)")
    #axes[2].grid(True, alpha=0.3)
    axes[2].legend()
    
    plt.tight_layout()
    save_plot(fig, "center_of_mass_divergence")
    plt.show()
    

    print(f"\nCenter of Mass Divergence Summary (Window Size: {window_size}):")
    print("=" * 60)
    
    pairs = [
        ("TAPSim (no MD) vs TAPSim with 5-atom MD", dist_12, ma_12),
        ("TAPSim (no MD) vs TAPSim with 10-atom MD", dist_13, ma_13),
        ("TAPSim with 5-atom MD vs TAPSim with 10-atom MD", dist_23, ma_23)
    ]
    
    for pair_name, raw_data, ma_data in pairs:
        print(f"\n{pair_name}:")
        raw_array = np.array(raw_data)
        ma_array = np.array(ma_data)
        
        # Print with higher precision
        print(f"  Raw Data     - Mean: {np.mean(raw_array):.6f} Å, Std: {np.std(raw_array):.6f} Å, Max: {np.max(raw_array):.6f} Å")
        print(f"  MA-{ma_window} Data  - Mean: {np.mean(ma_array):.6f} Å, Std: {np.std(ma_array):.6f} Å, Max: {np.max(ma_array):.6f} Å")
        print(f"  Raw Data     - Min: {np.min(raw_array):.6f} Å, Median: {np.median(raw_array):.6f} Å")
        print(f"  MA-{ma_window} Data  - Min: {np.min(ma_array):.6f} Å, Median: {np.median(ma_array):.6f} Å")
        
        # Percentile analysis
        p25_raw, p75_raw = np.percentile(raw_array, [25, 75])
        p25_ma, p75_ma = np.percentile(ma_array, [25, 75])
        print(f"  Raw Data     - 25th percentile: {p25_raw:.6f} Å, 75th percentile: {p75_raw:.6f} Å")
        print(f"  MA-{ma_window} Data  - 25th percentile: {p25_ma:.6f} Å, 75th percentile: {p75_ma:.6f} Å")
    
    from scipy import stats
    print(f"\nTrend Analysis (Linear Regression on Moving Averages):")
    print("-" * 50)
    
    for pair_name, _, ma_data in pairs:
        if len(ma_data) > 1:
            x_data = np.array(range(len(ma_data)))
            ma_array = np.array(ma_data)
            slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, ma_array)
            print(f"\n{pair_name}:")
            print(f"  Slope: {slope:.8f} Å/step")
            print(f"  R²: {r_value**2:.6f}")
            print(f"  P-value: {p_value:.6f}")
            print(f"  Standard Error: {std_err:.8f}")
            print(f"  Trend: {'Increasing' if slope > 0 else 'Decreasing' if slope < 0 else 'Flat'}")
            if abs(slope) > 1e-6:  # Lower threshold for meaningful changes
                print(f"  Change over 1000 steps: {slope * 1000:.6f} Å")
                print(f"  Change over 100 steps: {slope * 100:.6f} Å")

plot_center_of_mass_divergence(case1, case2, case3, window_size=10)


### Nearest Neighbor Matching

Match atoms by finding the closest atom in 3D space.

In [ ]:
def analyze_atomic_evaporation_timing(case1, case2, case3, n_atoms=500):
    """
    Quantify temporal shifts in evaporation patterns: When do the same atoms 
    get evaporated across different simulation cases?
    
    This analysis reveals how MD relaxation affects the sequence order of evaporation
    by tracking which specific atoms evaporate at different points in the simulation.
    
    Key Concepts:
    - Nth atom evaporated = N atoms removed from initial structure
    - Positive time difference = Case A evaporates atom later in sequence
    - Large differences (±100+ atoms) = Major reordering of evaporation sequence
    """
    
    def get_coordinate_ranges(case_data, n_atoms):
        """Extract coordinates for first n_atoms evaporation events"""
        coords = []
        for num_evap in sorted(case_data.keys())[:n_atoms]:
            df = case_data[num_evap]
            if len(df) > 0:
                x, y, z = df['x'].iloc[0], df['y'].iloc[0], df['z'].iloc[0]
                coords.append([x, y, z])
        return np.array(coords)
    
    def assign_atoms_to_locations(case_data, grid_size, n_atoms):
        """
        Assign each evaporation event to a spatial location using 3D grid.
        Records the FIRST time each location experiences evaporation.
        """
        location_timing = {}  # location_id -> first evaporation event (number of atoms evaporated)
        
        for num_evap in sorted(case_data.keys())[:n_atoms]:
            df = case_data[num_evap]
            if len(df) > 0:
                x, y, z = df['x'].iloc[0], df['y'].iloc[0], df['z'].iloc[0]
                
                # Convert coordinates to grid indices
                grid_x = int(x / grid_size) if grid_size > 0 else int(x * 1e6)
                grid_y = int(y / grid_size) if grid_size > 0 else int(y * 1e6)
                grid_z = int(z / grid_size) if grid_size > 0 else int(z * 1e6)
                location_id = (grid_x, grid_y, grid_z)
                
                # Record the FIRST time this location is evaporated
                if location_id not in location_timing:
                    location_timing[location_id] = num_evap
                    
        return location_timing
    
    def compare_atomic_timing(locations_a, locations_b, name_a, name_b):
        """Compare when the same atomic locations get evaporated between two cases"""
        matches = []
        common_locations = set(locations_a.keys()) & set(locations_b.keys())
        
        for location in common_locations:
            num_evap_a = locations_a[location]
            num_evap_b = locations_b[location]
            
            matches.append({
                'location': location,
                'num_evap_a': num_evap_a,
                'num_evap_b': num_evap_b,
                'time_diff': num_evap_a - num_evap_b,  # Positive = A evaporates later
                'earlier_num_evap': min(num_evap_a, num_evap_b),
                'case_a': name_a,
                'case_b': name_b
            })
        
        return matches

    print("="*80)
    print("TEMPORAL SHIFT ANALYSIS: Atomic Evaporation Timing")
    print("="*80)
    print("Analyzing when the same atomic locations evaporate across simulation methods...")
    print("\nEvaporation Interpretation:")
    print("- 0 atoms evaporated: Initial state (no atoms evaporated)")
    print("- N atoms evaporated: Nth atom evaporated (N atoms removed from structure)")
    print("- Time difference = when same atom location evaporates in different methods")
    
    # Get all coordinates to determine appropriate grid sizes
    coords1 = get_coordinate_ranges(case1, n_atoms)
    coords2 = get_coordinate_ranges(case2, n_atoms)
    coords3 = get_coordinate_ranges(case3, n_atoms)
    all_coords = np.vstack([coords1, coords2, coords3])
    
    # Calculate coordinate ranges
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    z_range = all_coords[:, 2].max() - all_coords[:, 2].min()
    max_range = max(x_range, y_range, z_range)
    
    print(f"\nCoordinate ranges: X={x_range:.2f}, Y={y_range:.2f}, Z={z_range:.2f} Å")
    print(f"Maximum coordinate range: {max_range:.2f} Å")
    
    print(f"\nTesting multiple grid sizes to find optimal atomic correspondence...")
    
    # Test multiple grid sizes from fine to coarse
    if max_range > 0:
        grid_sizes = [max_range/100, max_range/50, max_range/20, max_range/10, max_range/5]
    else:
        grid_sizes = [0.01, 0.1, 1.0]  # Fallback for zero range
    
    best_grid_size = None
    best_matches = None
    
    for grid_size in grid_sizes:
        print(f"\nTesting grid size: {grid_size:.2f} Å")
        
        # Get location timing for all cases
        locations_1 = assign_atoms_to_locations(case1, grid_size, n_atoms)
        locations_2 = assign_atoms_to_locations(case2, grid_size, n_atoms)
        locations_3 = assign_atoms_to_locations(case3, grid_size, n_atoms)
        
        # Compare all pairs
        matches_12 = compare_atomic_timing(locations_1, locations_2, "TAPSim", "TAPSim+5MD")
        matches_13 = compare_atomic_timing(locations_1, locations_3, "TAPSim", "TAPSim+10MD")
        matches_23 = compare_atomic_timing(locations_2, locations_3, "TAPSim+5MD", "TAPSim+10MD")
        
        total_matches = len(matches_12) + len(matches_13) + len(matches_23)
        overlap_percentage = total_matches / (3 * n_atoms) * 100
        
        print(f"  Unique locations: C1={len(locations_1)}, C2={len(locations_2)}, C3={len(locations_3)}")
        print(f"  Common locations: {len(matches_12)}, {len(matches_13)}, {len(matches_23)}")
        print(f"  Total matches: {total_matches} ({overlap_percentage:.1f}% atomic correspondence)")
        
        if total_matches > 10:  # Found meaningful overlap
            best_grid_size = grid_size
            best_matches = (matches_12, matches_13, matches_23)
            break
    
    if best_matches is None:
        print("\n" + "="*80)
        print("RESULT: COMPLETE ATOMIC DIVERGENCE DETECTED!")
        print("="*80)
        print(" - No meaningful atomic correspondence found at any resolution")
        print(" - MD relaxation fundamentally reorganizes which atoms evaporate")
        print(" - Different methods evaporate entirely different atoms")
        print(" - Cannot quantify temporal shifts - atomic divergence is complete")
        print("="*80)
        return None, None, None, None
    
    print(f"\n" + "="*80)
    print(f"ANALYSIS PROCEEDING WITH ATOMIC PRECISION: {best_grid_size:.2f} Å")
    print("="*80)
    return best_matches[0], best_matches[1], best_matches[2], best_grid_size


def plot_temporal_shift_analysis(matches_12, matches_13, matches_23, grid_size):
    """
    Visualize temporal shifts for evaporation of the same atomic locations.
    Shows when identical atoms evaporate across different methods.
    """
    pairs = [
        (matches_12, "TAPSim vs TAPSim with 5-atom MD"),
        (matches_13, "TAPSim vs TAPSim with 10-atom MD"),
        (matches_23, "TAPSim with 5-atom MD vs TAPSim with 10-atom MD"),
    ]
    
    fig, axes = plt.subplots(3, 1, figsize=(16, 14), sharex=True)

    for i, (matches, title) in enumerate(pairs):
        if not matches:
            axes[i].set_title(f"No common atoms: {title}")
            axes[i].text(0.5, 0.5, f"Complete atomic divergence\n(Precision: {grid_size:.2f} Å)", 
                        transform=axes[i].transAxes, ha='center', va='center', fontsize=12,
                        bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))
            continue
            
        # Extract timing data
        earlier_num_evap = [m['earlier_num_evap'] for m in matches]
        time_diffs = [m['time_diff'] for m in matches]
        
        # Create scatter plot with enhanced visualization
        scatter = axes[i].scatter(earlier_num_evap, time_diffs, 
                                s=50, alpha=0.7, c='steelblue', 
                                edgecolors='darkblue', linewidth=0.5)
        
        # Add reference lines for interpretation
        axes[i].axhline(y=0, color='red', linestyle='--', alpha=0.8, linewidth=2,
                       label='Same timing')
        axes[i].axhline(y=50, color='orange', linestyle=':', alpha=0.6, 
                       label='±50 atom difference')
        axes[i].axhline(y=-50, color='orange', linestyle=':', alpha=0.6)
        
        axes[i].set_ylabel("Time Difference (# of atoms)", fontsize=12)
        axes[i].set_title(f"{title} ({len(matches)} atoms)", fontsize=13)
        #axes[i].grid(True, alpha=0.4)
        axes[i].legend(loc='upper right')

    axes[-1].set_xlabel("Number of Atoms Evaporated", fontsize=12)

    plt.suptitle("Atomic Evaporation Timing Analysis", 
                 fontsize=15, fontweight='bold', y=0.98)
    plt.tight_layout()
    save_plot(fig, "temporal_shift_analysis")
    plt.show()

def print_temporal_shift_summary(matches_12, matches_13, matches_23, grid_size):
    """
    Print comprehensive quantitative summary of temporal shift analysis.
    Provides detailed interpretation of timing differences and their physical meaning.
    """
    pairs = [
        (matches_12, "TAPSim vs TAPSim with 5-atom MD"),
        (matches_13, "TAPSim vs TAPSim with 10-atom MD"),
        (matches_23, "TAPSim with 5-atom MD vs TAPSim with 10-atom MD"),
    ]
    
    print(f"\n" + "="*80)
    print(f"TEMPORAL SHIFT ANALYSIS SUMMARY (Atomic Precision: {grid_size:.2f} Å)")
    print("="*80)
    print("This analysis tracks WHEN the same atoms get evaporated")
    print("across different simulation methods, revealing sequence reordering effects.\n")
    
    print("INTERPRETATION GUIDE:")
    print(" - Positive timing difference: Case A evaporates atom later in sequence")
    print(" - Negative timing difference: Case A evaporates atom earlier in sequence")
    print(" - Large differences (±100+ atoms): Major reordering of evaporation sequence")
    print(" - High variability: Inconsistent/chaotic MD effects on timing")
    print("-"*80)

    for matches, title in pairs:
        print(f"\n{title}:")
        print("-" * len(title))
        
        if not matches:
            print("  No common atoms found - complete atomic divergence")
            print("  - MD fundamentally reorganizes which specific atoms evaporate")
            print("  - Cannot quantify timing effects due to atomic divergence")
            continue
            
        time_diffs = [m['time_diff'] for m in matches]
        
        # Basic statistics
        print(f"  STATISTICS:")
        print(f"     Common atoms analyzed: {len(matches)} out of 1000 ({len(matches)/1000*100:.1f}%)")
        print(f"     Mean timing difference: {np.mean(time_diffs):.2f} ± {np.std(time_diffs):.2f} atoms")
        print(f"     Median timing difference: {np.median(time_diffs):.2f} atoms")
        print(f"     Range: [{np.min(time_diffs):.0f}, {np.max(time_diffs):.0f}] atoms")
        print(f"     25th-75th percentile: [{np.percentile(time_diffs, 25):.0f}, {np.percentile(time_diffs, 75):.0f}] atoms")
        
        # Timing pattern analysis
        earlier = sum(1 for d in time_diffs if d > 0)
        later = sum(1 for d in time_diffs if d < 0)
        same = sum(1 for d in time_diffs if d == 0)
        
        case_a = matches[0]['case_a']
        case_b = matches[0]['case_b']
        
        print(f"\n  TIMING PATTERNS:")
        print(f"     {case_a} evaporates first: {later} atoms ({later/len(matches)*100:.1f}%)")
        print(f"     {case_b} evaporates first: {earlier} atoms ({earlier/len(matches)*100:.1f}%)")
        print(f"     Same timing: {same} atoms ({same/len(matches)*100:.1f}%)")
        
        mean_diff = np.mean(time_diffs)
        abs_mean = abs(mean_diff)
        
        print(f"\n  PHYSICAL INTERPRETATION:")
        if abs_mean > 50:
            direction = "LATER" if mean_diff > 0 else "EARLIER"
            print(f"     STRONG systematic effect: {case_a} evaporates atoms ~{abs_mean:.0f} atoms {direction}")
            if mean_diff > 0:
                print(f"     - {case_b} accelerates access to these atomic sites")
            else:
                print(f"     - {case_a} accelerates access to these atomic sites")
        elif abs_mean > 20:
            direction = "later" if mean_diff > 0 else "earlier"
            print(f"     Moderate systematic effect: {case_a} evaporates atoms ~{abs_mean:.0f} atoms {direction}")
        else:
            print(f"     No systematic timing bias detected (mean shift: {mean_diff:.1f} atoms)")
            
        # Consistency analysis
        std_diff = np.std(time_diffs)
        print(f"\n  CONSISTENCY ANALYSIS:")
        if std_diff > 100:
            print(f"     Very high timing variability (σ={std_diff:.0f}) - chaotic MD effects")
            print(f"     - MD introduces unpredictable, large-scale sequence reordering")
        elif std_diff > 50:
            print(f"     High timing variability (σ={std_diff:.0f}) - inconsistent MD effects")
            print(f"     - MD effects are significant but somewhat unpredictable")
        elif std_diff > 20:
            print(f"     Moderate timing variability (σ={std_diff:.0f}) - some MD inconsistency")
        else:
            print(f"     Low timing variability (σ={std_diff:.0f}) - consistent MD effects")
            
        # Sequence reordering magnitude
        large_shifts = sum(1 for d in time_diffs if abs(d) > 100)
        print(f"\n SEQUENCE REORDERING:")
        print(f"     Large shifts (>100 atoms): {large_shifts} atoms ({large_shifts/len(matches)*100:.1f}%)")
        if large_shifts > len(matches) * 0.3:
            print(f"     - Major evaporation sequence reorganization detected")
        elif large_shifts > 0:
            print(f"     - Moderate sequence reorganization in subset of atoms")
        else:
            print(f"     - Minor sequence adjustments only")

    print(f"\n" + "="*80)
    print("OVERALL CONCLUSIONS:")
    print("="*80)
    
    # Calculate overall atomic correspondence
    all_matches = len([m for matches, _ in pairs if matches for m in matches])
    total_possible = 3 * 1000  # 3 comparisons × 1000 atoms each
    overall_overlap = all_matches / total_possible * 100
    
    print(f"Overall atomic correspondence: {overall_overlap:.1f}% of atoms evaporate in multiple methods")
    
    if overall_overlap < 5:
        print("MASSIVE ATOMIC DIVERGENCE: MD fundamentally reorganizes evaporation")
        print("- MD is not a minor correction but a qualitative change in physics")
        print("- Different methods evaporate almost entirely different atoms")
    elif overall_overlap < 20:
        print("SIGNIFICANT ATOMIC DIVERGENCE: MD substantially alters evaporation patterns")
        print("- MD effects dominate over initial field-driven selection")
    else:
        print("- MODERATE ATOMIC CORRESPONDENCE: MD provides corrections to evaporation timing")
        
    # Systematic effect assessment
    systematic_effects = []
    for matches, title in pairs:
        if matches:
            time_diffs = [m['time_diff'] for m in matches]
            if abs(np.mean(time_diffs)) > 20:
                systematic_effects.append(title)
    
    if len(systematic_effects) >= 2:
        print(" STRONG SYSTEMATIC TIMING EFFECTS detected across multiple comparisons")
        print(" MD relaxation creates predictable acceleration/deceleration patterns")
    elif len(systematic_effects) == 1:
        print(" SELECTIVE SYSTEMATIC EFFECTS in specific method comparisons")
    else:
        print(" NO MAJOR SYSTEMATIC TIMING BIASES detected")
        
    print("="*80)

print("Analyzing temporal shifts in evaporation patterns...")
matches_12, matches_13, matches_23, grid_size = analyze_atomic_evaporation_timing(
    case1, case2, case3, n_atoms=1000)

if matches_12 is not None:
    # Create visualization
    plot_temporal_shift_analysis(matches_12, matches_13, matches_23, grid_size)
    
    # Print comprehensive summary
    print_temporal_shift_summary(matches_12, matches_13, matches_23, grid_size)
else:
    print("\n" + "="*80)
    print("FINAL CONCLUSION: COMPLETE ATOMIC DIVERGENCE!")
    print("="*80)
    print(" MD relaxation fundamentally reorganizes evaporation patterns")
    print(" No temporal shifts can be quantified - different atoms evaporate entirely")
    print(" This proves MD is essential for realistic APT simulation physics")
    print("="*80)


### Spatial Distribution of Evaporating Atoms (X-Y Projection):

Visualized as 2D heatmaps showing where atoms tend to evaporate from over time in each case.

In [ ]:
def plot_spatial_distribution(case1, case2, case3, step_range=None, bw_adjust=1.0):
    """
    Plots 2D KDE of spatial distribution of evaporated atoms for each case.

    Colorbar: Shows the relative spatial density of evaporated atom positions (KDE estimate),
    i.e., the finding an evaporated atom at a given (X, Y) location, in units of 1/m2.
    """
    if not step_range:
        step_range = range(min(case1.keys()), max(case1.keys())+1)

    def get_xy(case, steps):
        x, y = [], []
        for step in steps:
            if step in case:
                x.append(case[step]['x'].values[0])
                y.append(case[step]['y'].values[0])
        return x, y

    x1, y1 = get_xy(case1, step_range)
    x2, y2 = get_xy(case2, step_range)
    x3, y3 = get_xy(case3, step_range)

    fig, axs = plt.subplots(1, 3, figsize=(18, 5))
    cases = [
        (x1, y1, axs[0], 'Blues', 'Case 1 (TAPSim without MD)'),
        (x2, y2, axs[1], 'Reds',  'Case 2 (TAPSim with MD every 5 atoms)'),
        (x3, y3, axs[2], 'Greens','Case 3 (TAPSim with MD every 10 atoms)')
    ]
    for x, y, ax, cmap, title in cases:
        kde = sns.kdeplot(x=x, y=y, ax=ax, cmap=cmap, fill=True, thresh=0, levels=100, bw_adjust=bw_adjust)
        ax.set_title(title)
        ax.set_xlabel('X Position (m)')
        ax.set_ylabel('Y Position (m)')
        mappable = ax.collections[0]
        cbar = fig.colorbar(mappable, ax=ax)
        cbar.set_label('Relative Spatial Density (KDE, 1/m²)', rotation=270, labelpad=20)

    plt.tight_layout()
    save_plot(fig, f"spatial_distribution_cases_kde_bw{bw_adjust}")
    plt.show()
plot_spatial_distribution(case1, case2, case3, bw_adjust=1.0)


### Voltage Profile During Evaporation:
The voltage profile tracks the voltage changes over time for each case. 
The voltage is computed by rescaling the local potential with the evaporation field strength of the atoms at the field emitter. This voltage is not the potential in the computing mesh but a measurement rescaled to reflect the field strength at the emitter during evaporation.

In [ ]:
from scripts.voltage_profile import compare_voltage_profiles


In [ ]:
# Compare voltage profiles for each pair of cases
voltage_case1_2, voltage_case2_2, steps_case1_2 = compare_voltage_profiles(case1, case2)
voltage_case1_3, voltage_case2_3, steps_case1_3 = compare_voltage_profiles(case1, case3)
voltage_case2_3, voltage_case3_3, steps_case2_3 = compare_voltage_profiles(case2, case3)


In [ ]:
import pandas as pd

window_size = 50

# Convert to pandas Series for moving average
voltage_case1_2_ma = pd.Series(voltage_case1_2).rolling(window=window_size, min_periods=1, center=True).mean()
voltage_case2_2_ma = pd.Series(voltage_case2_2).rolling(window=window_size, min_periods=1, center=True).mean()
voltage_case3_3_ma = pd.Series(voltage_case3_3).rolling(window=window_size, min_periods=1, center=True).mean()

plt.figure(figsize=(10, 6))

# Plot raw (lighter)
plt.plot(steps_case1_2[:989], voltage_case1_2[:989], label='Case 1: TAPSim without MD (Raw)', color='blue', alpha=0.2)
plt.plot(steps_case1_2[:989], voltage_case2_2[:989], label='Case 2: TAPSim with 5-atom MD (Raw)', color='red', alpha=0.2)
plt.plot(steps_case1_3[:989], voltage_case3_3[:989], label='Case 3: TAPSim with 10-atom MD (Raw)', color='green', alpha=0.2)

# Plot moving average (darker)
plt.plot(steps_case1_2[:989], voltage_case1_2_ma[:989], label=f'Case 1: TAPSim without MD (MA-{window_size})', color='blue', linewidth=2)
plt.plot(steps_case1_2[:989], voltage_case2_2_ma[:989], label=f'Case 2: TAPSim with 5-atom MD (MA-{window_size})', color='red', linewidth=2)
plt.plot(steps_case1_3[:989], voltage_case3_3_ma[:989], label=f'Case 3: TAPSim with 10-atom MD (MA-{window_size})', color='green', linewidth=2)

plt.xlabel("Number of Atoms Evaporated")
plt.ylabel("Avg Voltage")
plt.ylim(5, 13)
plt.title(f"Voltage Profile Comparison: Case 1, Case 2, Case 3 (Moving Average, Window={window_size})")
plt.legend()
plt.tight_layout()
save_plot(plt.gcf(), "voltage_profile_comparison")
plt.show()


## Evaporation probabilities

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def extract_selection_metrics(batches_case1, batches_case2, batches_case3):
    """
    Extracts only probability for each step from the batch dictionaries for Case 1, Case 2, and Case 3.
    Returns a DataFrame with step-wise probability metrics for each case.
    """
    data_case1 = []
    data_case2 = []
    data_case3 = []

    # Extract metrics for Case 1
    for step, df in batches_case1.items():
        row = df.iloc[0]
        data_case1.append({
            'step': step,
            'probability': row['probability']
        })
    
    # Extract metrics for Case 2
    for step, df in batches_case2.items():
        row = df.iloc[0]
        data_case2.append({
            'step': step,
            'probability': row['probability']
        })

    # Extract metrics for Case 3
    for step, df in batches_case3.items():
        row = df.iloc[0]
        data_case3.append({
            'step': step,
            'probability': row['probability']
        })

    # Combine the data into DataFrames
    df_case1 = pd.DataFrame(data_case1)
    df_case2 = pd.DataFrame(data_case2)
    df_case3 = pd.DataFrame(data_case3)

    return df_case1, df_case2, df_case3

def plot_selection_comparison(df_case1, df_case2, df_case3, window_size=50):
    """
    Plots comparison of evaporation probability for Case 1 vs. Case 2 vs. Case 3,
    with optional moving average smoothing.
    """
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))

    # Compute moving averages
    ma1 = df_case1['probability'].rolling(window=window_size, min_periods=1, center=True).mean()
    ma2 = df_case2['probability'].rolling(window=window_size, min_periods=1, center=True).mean()
    ma3 = df_case3['probability'].rolling(window=window_size, min_periods=1, center=True).mean()

    # Plot raw probabilities (lighter)
    ax.plot(df_case1['step'], df_case1['probability'], label='Case 1: TAPSim without MD (Raw)', color='blue', alpha=0.2)
    ax.plot(df_case2['step'], df_case2['probability'], label='Case 2: TAPSim with 5-atom MD (Raw)', color='red', alpha=0.2)
    ax.plot(df_case3['step'], df_case3['probability'], label='Case 3: TAPSim with 10-atom MD (Raw)', color='green', alpha=0.2)

    # Plot moving averages (darker)
    ax.plot(df_case1['step'], ma1, label=f'Case 1: TAPSim without MD (MA-{window_size})', color='blue', linewidth=2)
    ax.plot(df_case2['step'], ma2, label=f'Case 2: TAPSim with 5-atom MD (MA-{window_size})', color='red', linewidth=2)
    ax.plot(df_case3['step'], ma3, label=f'Case 3: TAPSim with 10-atom MD (MA-{window_size})', color='green', linewidth=2)

    ax.set_ylabel('Evaporation Probability')
    ax.set_xlabel('Number of Atoms Evaporated')
    ax.set_ylim(0.00030, 0.00055)
    ax.set_title(f'Evaporation Probability Comparison Across Cases (Moving Average, Window={window_size})')
    ax.legend()
    #ax.grid(True, alpha=0.3)

    plt.tight_layout()
    save_plot(fig, "evaporation_probability_comparison")
    plt.show()


In [ ]:
# Compute selection metrics for all three approaches (Case 1, Case 2, Case 3)
df_sel_case1, df_sel_case2, df_sel_case3 = extract_selection_metrics(case1, case2, case3)

# Plot direct comparisons
plot_selection_comparison(df_sel_case1, df_sel_case2, df_sel_case3)


## Emitter Surface Z-Height Distribution

### Moving Average on Z Height

In [ ]:
# Extract and plot apex Z-height directly from the three cases
print("Extracting apex Z-height data...")

def extract_apex_z(case_data):
    """Extract apex Z coordinates from case data"""
    steps = []
    apex_z = []
    
    for step in sorted(case_data.keys()):
        df = case_data[step]
        if len(df) > 0:
            steps.append(step)
            apex_z.append(df['apexZ'].iloc[0])  # Extract apexZ directly
    
    return steps, apex_z

# Extract data for all three cases
steps_1, apex_z_1 = extract_apex_z(case1)
steps_2, apex_z_2 = extract_apex_z(case2)
steps_3, apex_z_3 = extract_apex_z(case3)

# Find common length for comparison
min_len = min(len(steps_1), len(steps_2), len(steps_3))
print(f"Using {min_len} common steps for analysis")

# Trim to common length
steps_common = steps_1[:min_len]
apex_z_1_trim = apex_z_1[:min_len]
apex_z_2_trim = apex_z_2[:min_len]
apex_z_3_trim = apex_z_3[:min_len]

# Convert to nanometers for better readability
apex_z_1_nm = [z * 1e9 for z in apex_z_1_trim]
apex_z_2_nm = [z * 1e9 for z in apex_z_2_trim]
apex_z_3_nm = [z * 1e9 for z in apex_z_3_trim]

# Check initial values
print(f"Initial apex Z-heights:")
print(f"  Case 1: {apex_z_1_nm[0]:.6f} nm")
print(f"  Case 2: {apex_z_2_nm[0]:.6f} nm")
print(f"  Case 3: {apex_z_3_nm[0]:.6f} nm")

# Apply moving average
window_size = 50
import pandas as pd

apex_z_1_smooth = pd.Series(apex_z_1_nm).rolling(window=window_size, min_periods=1).mean()
apex_z_2_smooth = pd.Series(apex_z_2_nm).rolling(window=window_size, min_periods=1).mean()
apex_z_3_smooth = pd.Series(apex_z_3_nm).rolling(window=window_size, min_periods=1).mean()

# Calculate shrinkage (current - initial), so shrinkage is negative as apex decreases
apex_z_1_shrink_inv = [z - apex_z_1_nm[0] for z in apex_z_1_nm]
apex_z_2_shrink_inv = [z - apex_z_2_nm[0] for z in apex_z_2_nm]
apex_z_3_shrink_inv = [z - apex_z_3_nm[0] for z in apex_z_3_nm]

# Moving average for shrinkage
apex_z_1_shrink_inv_smooth = pd.Series(apex_z_1_shrink_inv).rolling(window=window_size, min_periods=1).mean()
apex_z_2_shrink_inv_smooth = pd.Series(apex_z_2_shrink_inv).rolling(window=window_size, min_periods=1).mean()
apex_z_3_shrink_inv_smooth = pd.Series(apex_z_3_shrink_inv).rolling(window=window_size, min_periods=1).mean()

plt.figure(figsize=(12, 6))

# Plot raw shrinkage (lighter)
plt.plot(steps_common, apex_z_1_shrink_inv, label='Case 1: TAPSim without MD (Raw)', color='blue', alpha=0.2)
plt.plot(steps_common, apex_z_2_shrink_inv, label='Case 2: TAPSim with 5-atom MD (Raw)', color='red', alpha=0.2)
plt.plot(steps_common, apex_z_3_shrink_inv, label='Case 3: TAPSim with 10-atom MD (Raw)', color='green', alpha=0.2)

# Plot smoothed shrinkage
plt.plot(steps_common, apex_z_1_shrink_inv_smooth, label=f'Case 1: TAPSim without MD (MA-{window_size})', color='blue', linewidth=2)
plt.plot(steps_common, apex_z_2_shrink_inv_smooth, label=f'Case 2: TAPSim with 5-atom MD (MA-{window_size})', color='red', linewidth=2)
plt.plot(steps_common, apex_z_3_shrink_inv_smooth, label=f'Case 3: TAPSim with 10-atom MD (MA-{window_size})', color='green', linewidth=2)

plt.title(f"Emitter Apex Z-Height Evolution (Moving Average, Window={window_size})")
plt.xlabel("Number of Atoms Evaporated")
plt.ylabel("Apex Shrinkage (nm, negative = shrinkage)")
plt.ylim(-0.5, 0.05)
plt.legend()
plt.tight_layout()
save_plot(plt.gcf(), f"apex_shrinkage_evolution_negative_{window_size}")
plt.show()


In [ ]:
# Distribution profile analysis of apex Z-heights - KDE and Box plots only
print("Creating apex Z-height distribution profiles...")

# Create figure with subplots for KDE and box plots only
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 1. Kernel Density Estimation (KDE) plots
import seaborn as sns
sns.kdeplot(data=apex_z_1_nm, ax=axes[0], color='blue', label='Case 1: TAPSim only', linewidth=2)
sns.kdeplot(data=apex_z_2_nm, ax=axes[0], color='red', label='Case 2: TAPSim with 5-atom MD', linewidth=2)
sns.kdeplot(data=apex_z_3_nm, ax=axes[0], color='green', label='Case 3: TAPSim with 10-atom MD', linewidth=2)
axes[0].set_xlabel('Apex Z-Height (nm)')
axes[0].set_ylabel('Density')
axes[0].set_title('Apex Height Distribution (KDE)')
axes[0].legend()
#axes[0].grid(True, alpha=0.3)

# 2. Box plots for comparison
box_data = [apex_z_1_nm, apex_z_2_nm, apex_z_3_nm]
box_labels = ['Case 1\n(TAPSim only)', 'Case 2\n(TAPSim with 5-atom MD)', 'Case 3\n(TAPSim with 10-atom MD)']
box_colors = ['lightblue', 'lightcoral', 'lightgreen']

bp = axes[1].boxplot(box_data, labels=box_labels, patch_artist=True, 
                    showmeans=True, meanline=True)
for patch, color in zip(bp['boxes'], box_colors):
    patch.set_facecolor(color)
axes[1].set_ylabel('Apex Z-Height (nm)')
axes[1].set_title('Apex Height Distribution (Box Plot)')
#axes[1].grid(True, alpha=0.3)

plt.tight_layout()
save_plot(fig, "apex_height_distribution_kde_box")
plt.show()

# Print detailed statistical summary
print(f"\nDetailed Statistical Summary of Apex Heights:")
print("=" * 70)

import scipy.stats as stats

cases_data = [
    ("Case 1 (TAPSim only)", apex_z_1_nm, 'blue'),
    ("Case 2 (TAPSim + MD 5 atoms)", apex_z_2_nm, 'red'),
    ("Case 3 (TAPSim + MD 10 atoms)", apex_z_3_nm, 'green')
]

for case_name, data, color in cases_data:
    print(f"\n{case_name}:")
    print("-" * len(case_name))
    
    # Basic statistics
    print(f"  Count:           {len(data)}")
    print(f"  Mean:            {np.mean(data):.6f} nm")
    print(f"  Median:          {np.median(data):.6f} nm")
    print(f"  Std Deviation:   {np.std(data):.6f} nm")
    print(f"  Variance:        {np.var(data):.6f} nm²")
    print(f"  Min:             {np.min(data):.6f} nm")
    print(f"  Max:             {np.max(data):.6f} nm")
    print(f"  Range:           {np.max(data) - np.min(data):.6f} nm")
    
    # Percentiles
    percentiles = [5, 25, 75, 95]
    percs = np.percentile(data, percentiles)
    print(f"  5th percentile:  {percs[0]:.6f} nm")
    print(f"  25th percentile: {percs[1]:.6f} nm")
    print(f"  75th percentile: {percs[2]:.6f} nm")
    print(f"  95th percentile: {percs[3]:.6f} nm")
    print(f"  IQR:             {percs[2] - percs[1]:.6f} nm")
    
    # Shape statistics
    skewness = stats.skew(data)
    kurtosis = stats.kurtosis(data)
    print(f"  Skewness:        {skewness:.6f}")
    print(f"  Kurtosis:        {kurtosis:.6f}")
    
    # Distribution shape interpretation
    if abs(skewness) < 0.5:
        skew_desc = "approximately symmetric"
    elif skewness > 0.5:
        skew_desc = "right-skewed (tail extends to higher values)"
    else:
        skew_desc = "left-skewed (tail extends to lower values)"
    
    print(f"  Distribution:    {skew_desc}")

# Statistical comparison between cases
print(f"\n" + "=" * 70)
print("STATISTICAL COMPARISONS:")
print("=" * 70)

# Perform statistical tests
from scipy import stats

# Compare Case 1 vs Case 2
ks_stat_12, ks_p_12 = stats.ks_2samp(apex_z_1_nm, apex_z_2_nm)
t_stat_12, t_p_12 = stats.ttest_ind(apex_z_1_nm, apex_z_2_nm)

# Compare Case 1 vs Case 3
ks_stat_13, ks_p_13 = stats.ks_2samp(apex_z_1_nm, apex_z_3_nm)
t_stat_13, t_p_13 = stats.ttest_ind(apex_z_1_nm, apex_z_3_nm)

# Compare Case 2 vs Case 3
ks_stat_23, ks_p_23 = stats.ks_2samp(apex_z_2_nm, apex_z_3_nm)
t_stat_23, t_p_23 = stats.ttest_ind(apex_z_2_nm, apex_z_3_nm)

comparisons = [
    ("Case 1 vs Case 2", ks_stat_12, ks_p_12, t_stat_12, t_p_12),
    ("Case 1 vs Case 3", ks_stat_13, ks_p_13, t_stat_13, t_p_13),
    ("Case 2 vs Case 3", ks_stat_23, ks_p_23, t_stat_23, t_p_23)
]

for comp_name, ks_stat, ks_p, t_stat, t_p in comparisons:
    print(f"\n{comp_name}:")
    print(f"  Kolmogorov-Smirnov Test:")
    print(f"    Statistic: {ks_stat:.6f}")
    print(f"    P-value:   {ks_p:.6e}")
    print(f"    Result:    {'Significantly different' if ks_p < 0.05 else 'Not significantly different'} distributions")
    
    print(f"  T-test (means):")
    print(f"    Statistic: {t_stat:.6f}")
    print(f"    P-value:   {t_p:.6e}")
    print(f"    Result:    {'Significantly different' if t_p < 0.05 else 'Not significantly different'} means")

print(f"\n" + "=" * 70)
